In [1]:
# STANDARD LIBRARIES
import pandas as pd
import numpy as np
import pickle

# VISUALS
import matplotlib.pyplot as plt
import seaborn as sns

# FEATURE ENGINEERING AND PREPROCESSING
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split

# MODELING
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

# METRICS
from sklearn.metrics import confusion_matrix

In [3]:
main = pd.read_csv("../data/clean-data/main-engineered.csv")
main.drop(columns=["Unnamed: 0"], inplace=True)

In [5]:
X = main.drop(columns=[
    "label_zip",
#     "label_street",
#     "zipcode",
    "zip_street"
#     "zip_num_street"
])
y = main["label_zip"]
# y = main["label_street"]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=42)

# ss = StandardScaler()
# 
# Z = ss.fit_transform(X)
# Z_train =  ss.fit_transform(X_train)
# Z_test = ss.transform(X_test)

In [6]:
y.value_counts()

15    9071
23    8478
27    7688
22    7015
21    6674
18    6666
19    6164
26    5578
16    5354
11    4572
14    4391
17    4044
13    3734
20    3069
9     2400
25    2284
10    2276
12    1412
8     1357
7     1192
6      934
31     858
1      676
4      475
2      441
34     231
3      224
30     215
28      18
32      13
33       8
24       1
29       1
5        1
Name: label_zip, dtype: int64

In [89]:
logreg_pipe = Pipeline([
    ("ss", StandardScaler()),
    ("logreg", LogisticRegression())
])

logreg_pipe.get_params()

{'memory': None,
 'steps': [('ss', StandardScaler(copy=True, with_mean=True, with_std=True)),
  ('logreg',
   LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                      intercept_scaling=1, l1_ratio=None, max_iter=100,
                      multi_class='auto', n_jobs=None, penalty='l2',
                      random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                      warm_start=False))],
 'verbose': False,
 'ss': StandardScaler(copy=True, with_mean=True, with_std=True),
 'logreg': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 'ss__copy': True,
 'ss__with_mean': True,
 'ss__with_std': True,
 'logreg__C': 1.0,
 'logreg__class_weight': None,
 'logre

### Model 1 

In [90]:
params_1 ={
#     "logreg__C": [0.01, 0.1, 1],
#     "logreg__penalty": ["l1", "l2"],
#     "logreg__verbose": [10]
}

In [91]:
gs_1 = GridSearchCV(
    logreg_pipe,
    params_q,
    cv=5
)

In [ ]:
gs_1.fit(X_train, y_train)

/home/patrick/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [83]:
model_1_params = gs.best_params_

model_1_best_score = gs.best_score_

model_1_cv_results = gs.cv_results_

model_1_train_score = gs.score(X_train, y_train)

model_1_test_score = gs.score(X_test, y_test)

model_1_df = pd.DataFrame(gs.cv_results_)

model_1_preds = gs.predict(X_test)

model_1_cm = confusion_matrix(y_test, logreg_preds)

model_1_cm_df = pd.DataFrame(logreg_cm)

print(model_2_train_score)
print(model_2_test_score)

model_2_params

### Model 2

In [90]:
params_2 = {
    "logreg__C": [0.01, 0.1, 1],
    "logreg__penalty": ["l1", "l2"],
    "logreg__verbose": [10]
}

In [91]:
gs_2 = GridSearchCV(
    logreg_pipe,
    params,
    cv=5
)

In [ ]:
gs_2.fit(X_train, y_train)

/home/patrick/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
model_2_params = gs.best_params_

model_2_best_score = gs.best_score_

model_2_cv_results = gs.cv_results_

model_2_train_score = gs.score(X_train, y_train)

model_2_test_score = gs.score(X_test, y_test)

model_2_df = pd.DataFrame(gs.cv_results_)

model_2_preds = gs.predict(X_test)

model_2_cm = confusion_matrix(y_test, logreg_preds)

model_2_cm_df = pd.DataFrame(logreg_cm)

print(model_2_train_score)
print(model_2_test_score)

model_2_params

### Model 3

In [90]:
params_3 = {
    "logreg__C": [0.01, 0.1, 1],
    "logreg__solver": ["saga"],
    "logreg__penalty": ["l1", "l2"],
    "logreg__verbose": [10]
}

In [91]:
gs_3 = GridSearchCV(
    logreg_pipe,
    params,
    cv=5
)

In [92]:
gs_3.fit(X_train, y_train)

/home/patrick/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 249 seconds


/home/patrick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 256 seconds


/home/patrick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.3min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.3min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

max_iter reached after 247 seconds


/home/patrick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 198 seconds


/home/patrick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.3min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.3min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

In [95]:
# # model_3_params = gs.best_params_

# # model_3_best_score = gs.best_score_

# model_3_cv_results = gs.cv_results_

# model_3_train_score = gs.score(X_train, y_train)

# model_3_test_score = gs.score(X_test, y_test)

# model_3_df = pd.DataFrame(gs.cv_results_)

# model_3_preds = gs.predict(X_test)

# model_3_cm = confusion_matrix(y_test, logreg_preds)

# model_3_cm_df = pd.DataFrame(logreg_cm)

# print(model_3_train_score)
# print(model_3_test_score)

# model_3_params

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'